In [1]:
import pandas as pd
import os
import datetime
import numpy as np
import gc

os.getcwd()

# To check about the 2019 weekly file path df week filter for the rolling later

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration'

In [2]:
prior_quarter_End=datetime.date(2019,2,2)
recent_complete_quarter_End=prior_quarter_End+datetime.timedelta(days=7*13)

print("prior_quarter_End",prior_quarter_End)
print("recent_complete_quarter_End",recent_complete_quarter_End)

lapsed_date_begin=prior_quarter_End-datetime.timedelta(days=(4*365+1+6))
print("lapsed_date_begin",lapsed_date_begin)


prior_quarter_End 2019-02-02
recent_complete_quarter_End 2019-05-04
lapsed_date_begin 2015-01-27


In [3]:
lapsed=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed','transaction_date'],dtype=str).drop_duplicates()
lapsed=lapsed[lapsed['transaction_date']>=str(lapsed_date_begin)]

print(lapsed['transaction_date'].min())
print(lapsed['transaction_date'].max())

2015-01-27
2017-02-26


In [4]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date']):
    chunk = chunk.drop_duplicates()
    chunk=chunk[chunk['transaction_date']>=str(lapsed_date_begin)]
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()    

1 2019-05-17 19:03:46.105521
2 2019-05-17 19:04:14.810625
3 2019-05-17 19:04:39.484599
4 2019-05-17 19:05:07.969745
5 2019-05-17 19:05:38.583563
6 2019-05-17 19:06:15.225318
7 2019-05-17 19:06:48.903501
8 2019-05-17 19:07:28.639997
9 2019-05-17 19:08:11.031257
10 2019-05-17 19:08:54.221631
11 2019-05-17 19:09:39.563078
12 2019-05-17 19:10:32.199946
13 2019-05-17 19:11:25.029636
14 2019-05-17 19:12:05.466368
15 2019-05-17 19:12:41.792224


In [5]:
all_rewards_most_recent=dftrans_before_20180922.append(lapsed)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
print(all_rewards_most_recent.shape)

del chunk
del dftrans_before_20180922
gc.collect()

(23042758, 2)


126

In [5]:
# cell [6]
import os
import pandas as pd
import datetime

def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

In [6]:
# Before the end of Q4 only + 1st week in 2019Q1           
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recursive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,2,9)] # hard-code due to the 1st week of daily in item available
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)] # hard-code
print(historical_daily_df.shape)

(20, 2)


In [8]:
df_historical_daily_20180929_to_before_Q=historical_daily_df[historical_daily_df['week_end_dt']<=prior_quarter_End]
df_historical_daily_in_recent_Q=historical_daily_df[(historical_daily_df['week_end_dt']>prior_quarter_End) & (historical_daily_df['week_end_dt']<=recent_complete_quarter_End)]
print(df_historical_daily_20180929_to_before_Q.shape)
print(df_historical_daily_in_recent_Q.shape)


(19, 2)
(1, 2)


In [9]:
weekly_2019_daily_df=list(recrusive_file_gen("/home/jian/BigLots/"))

weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2018" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2017" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "2016" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "Historical" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "hist_daily_data_itemlevel_decompressed" not in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if ".txt" in x]
weekly_2019_daily_df=[x for x in weekly_2019_daily_df if "aily" in x]
weekly_2019_daily_df=pd.DataFrame({"file_path":weekly_2019_daily_df})
weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])

weekly_2019_daily_df['week_end_dt']=weekly_2019_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
print(weekly_2019_daily_df.shape)
weekly_2019_daily_df.head(2)

weekly_2019_daily_df_in_recent_Q=weekly_2019_daily_df[(weekly_2019_daily_df['week_end_dt']>datetime.date(2019,2,9)) & (weekly_2019_daily_df['week_end_dt']<=recent_complete_quarter_End)]
# Change to > prior_quarter_End for the next quarter
print(weekly_2019_daily_df_in_recent_Q.shape)


weekly_2019_daily_df_before_Q=weekly_2019_daily_df[weekly_2019_daily_df['week_end_dt']<=prior_quarter_End]
weekly_2019_daily_df_before_Q=weekly_2019_daily_df_before_Q[weekly_2019_daily_df_before_Q['week_end_dt']>df_historical_daily_20180929_to_before_Q['week_end_dt'].max()]
print(weekly_2019_daily_df_before_Q.shape) # Q1 should be 0, and Q2&after not


(19, 2)
(12, 2)
(0, 2)


In [10]:
df_all_daily_files_20190922_before_quarter=df_historical_daily_20180929_to_before_Q.append(weekly_2019_daily_df_before_Q)

df_all_daily_files_in_recent_quarter=df_historical_daily_in_recent_Q.append(weekly_2019_daily_df_in_recent_Q)


In [11]:
rew_df_after_20180929_before_quarter=pd.DataFrame()
rew_df_after_recent_quarter_only=pd.DataFrame()


for file in df_all_daily_files_20190922_before_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_20180929_before_quarter=rew_df_after_20180929_before_quarter.append(df)
    print(file,datetime.datetime.now())

for file in df_all_daily_files_in_recent_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",dtype=str,usecols=["customer_id_hashed","transaction_dt"]).drop_duplicates().rename(columns={"transaction_dt":"transaction_date"})
    df=df[~pd.isnull(df['customer_id_hashed'])]
    rew_df_after_recent_quarter_only=rew_df_after_recent_quarter_only.append(df)
    print(file,datetime.datetime.now())


/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20180929.txt 2019-05-17 19:33:27.123825
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181006.txt 2019-05-17 19:33:57.800787
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181013.txt 2019-05-17 19:34:19.644532
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181020.txt 2019-05-17 19:34:42.975700
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt 2019-05-17 19:35:07.116502
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt 2019-05-17 19:35:32.906771
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt 2019-05-17 19:35:59.973582
/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt 2019-05-17 19:36:32.383288


In [12]:
all_rewards_most_recent=all_rewards_most_recent.append(rew_df_after_20180929_before_quarter)
all_rewards_most_recent=all_rewards_most_recent.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent=all_rewards_most_recent.drop_duplicates(['customer_id_hashed'])
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date']

all_rewards_most_recent_After=all_rewards_most_recent[['customer_id_hashed','transaction_date']].append(rew_df_after_recent_quarter_only)
all_rewards_most_recent_After=all_rewards_most_recent_After.sort_values(["customer_id_hashed","transaction_date"],ascending=[True,False])
all_rewards_most_recent_After=all_rewards_most_recent_After.drop_duplicates(['customer_id_hashed']).rename(columns={"transaction_date":"transaction_date_after_the_Quarter"})

all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_rewards_most_recent_After,on="customer_id_hashed",how="outer")
del all_rewards_most_recent_After

gc.collect()

79

In [13]:
rew_df_after_20180929_before_quarter.tail(2)

,transaction_date,customer_id_hashed
11377066,2019-02-02,67e696e5c360783a8d7f56bd9f97f423a6385027443be0...
11377126,2019-01-27,fce32ce61bd38b4e23d7f88a6528ebe404502350fd6fc5...


In [14]:
rew_df_after_recent_quarter_only.tail(2)

,transaction_date,customer_id_hashed
12959879,2019-05-03,da09831b5743ecd5b42ffc59c4155df6ff2ea8a320be94...
12960022,2019-05-04,fd437b6358f7476efbd4af9c423a7ca7cff605f1a4e054...


In [15]:
all_rewards_most_recent['transaction_date_before_the_Quarter']=all_rewards_most_recent['transaction_date_before_the_Quarter'].fillna("2099-12-31")
all_rewards_most_recent['transaction_date_before_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_before_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_pre_Quarter_End']=prior_quarter_End-all_rewards_most_recent['transaction_date_before_the_Quarter']
# Filled the na before with a later adte 2019-12-31


all_rewards_most_recent['transaction_date_after_the_Quarter']=pd.to_datetime(all_rewards_most_recent['transaction_date_after_the_Quarter'],format='%Y-%m-%d')
all_rewards_most_recent['Days_to_recentt_Quarter_End']=recent_complete_quarter_End-all_rewards_most_recent['transaction_date_after_the_Quarter']
print(datetime.datetime.now())

2019-05-17 20:11:01.216482


In [16]:
all_rewards_most_recent['Month_to_pre_Quarter_End']=all_rewards_most_recent['Days_to_pre_Quarter_End'].apply(lambda x: x.days/(365.25/12))
all_rewards_most_recent['Month_to_recent_Quarter_End']=all_rewards_most_recent['Days_to_recentt_Quarter_End'].apply(lambda x: x.days/(365.25/12))
print(datetime.datetime.now())

2019-05-17 20:20:52.103768


In [17]:
all_rewards_most_recent['Group_before_the_Quarter']=np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>12) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_pre_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_pre_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_pre_Quarter_End']<0,"NotAvailable_Before_Quarter","NaN")
                                                                    )
                                                           )
                                                   )
all_rewards_most_recent['Group_after_the_Quarter']=np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>12) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=48),"Lapsed_13_48",
                                                   np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']>48,"WD_48+",
                                                            np.where((all_rewards_most_recent['Month_to_recent_Quarter_End']>=0) & (all_rewards_most_recent['Month_to_recent_Quarter_End']<=12),"Active",
                                                                    np.where(all_rewards_most_recent['Month_to_recent_Quarter_End']<0,"NotAvailable_Before_Q4","NaN")
                                                                    )
                                                           )
                                                   )

In [18]:
all_rewards_most_recent['Group_before_the_Quarter'].unique().tolist()

['Active', 'Lapsed_13_48', 'WD_48+', 'NotAvailable_Before_Quarter']

In [19]:
all_rewards_most_recent['Group_after_the_Quarter'].unique()

array(['Active', 'Lapsed_12_48', 'WD_48+'], dtype=object)

In [20]:
RecentQuarter_sales_by_id=pd.DataFrame()

for file in df_all_daily_files_in_recent_quarter['file_path'].tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df.groupby(['customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index()
    
    RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.append(df)
    print(file,datetime.datetime.now())
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.groupby("customer_id_hashed")['item_transaction_amt'].sum().to_frame().reset_index()
RecentQuarter_sales_by_id=RecentQuarter_sales_by_id.rename(columns={"item_transaction_amt":"sales_recent_Quarter"})
RecentQuarter_sales_by_id['Recent_Quarter_Shopping_Group']="Recent_Quarter_Shopped"

/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt 2019-05-17 20:21:51.708929
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt 2019-05-17 20:22:23.623408
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormDailySales20190226-112611-940.txt 2019-05-17 20:22:53.392344
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormDailySales20190305-112738-899.txt 2019-05-17 20:23:25.715889
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormDailySales20190312-121205-996.txt 2019-05-17 20:23:56.735460
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormDailySales20190319-112334-057.txt 2019-05-17 20:24:27.642712
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormDailySales20190326-112740-903.txt 2019-05-17 20:25:00.839744
/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-30/MediaStormDailySales20190402-112556-551.txt 2019-05-17 20:25:33.

In [21]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,RecentQuarter_sales_by_id,on="customer_id_hashed",how="outer")
all_rewards_most_recent.shape

(27017065, 12)

In [22]:
Recent_Quarter_new_sign_ups_files=list(recrusive_file_gen("/home/jian/BigLots/"))

Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "Master" in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2018" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2017" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if "2016" not in x]
Recent_Quarter_new_sign_ups_files=[x for x in Recent_Quarter_new_sign_ups_files if ".txt" in x]

Recent_Quarter_new_sign_ups_files_df=pd.DataFrame({"file_path":Recent_Quarter_new_sign_ups_files})
Recent_Quarter_new_sign_ups_files_df['week_end_dt']=Recent_Quarter_new_sign_ups_files_df['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10])
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']<=str(recent_complete_quarter_End)]
Recent_Quarter_new_sign_ups_files_df=Recent_Quarter_new_sign_ups_files_df[Recent_Quarter_new_sign_ups_files_df['week_end_dt']>str(prior_quarter_End)]


In [23]:
all_new_sign_ups=pd.DataFrame()
for file in Recent_Quarter_new_sign_ups_files_df['file_path'].unique().tolist():
    df=pd.read_table(file,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    all_new_sign_ups=all_new_sign_ups.append(df)
print(all_new_sign_ups.shape)

all_new_sign_ups['NewRewards']="Recent_Quarter_New_Sign_Ups"
all_new_sign_ups=all_new_sign_ups.drop_duplicates("customer_id_hashed")

(1070123, 2)


In [24]:
all_rewards_most_recent=pd.merge(all_rewards_most_recent,all_new_sign_ups,on="customer_id_hashed",how="outer")
all_rewards_most_recent['NewRewards']=all_rewards_most_recent['NewRewards'].fillna("Old_Rewards_Members")

all_rewards_most_recent['sales_recent_Quarter']=all_rewards_most_recent['sales_recent_Quarter'].fillna(0)
all_rewards_most_recent['Recent_Quarter_Shopping_Group']=all_rewards_most_recent['Recent_Quarter_Shopping_Group'].fillna("Recent_Quarter_No_Shop")

all_rewards_most_recent.shape


(27030825, 14)

In [25]:
all_rewards_most_recent.columns.tolist()

['customer_id_hashed',
 'transaction_date',
 'transaction_date_before_the_Quarter',
 'transaction_date_after_the_Quarter',
 'Days_to_pre_Quarter_End',
 'Days_to_recentt_Quarter_End',
 'Month_to_pre_Quarter_End',
 'Month_to_recent_Quarter_End',
 'Group_before_the_Quarter',
 'Group_after_the_Quarter',
 'sales_recent_Quarter',
 'Recent_Quarter_Shopping_Group',
 'sign_up_date',
 'NewRewards']

In [26]:
output_id_count=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['customer_id_hashed'].count().to_frame().reset_index().rename(columns={"customer_id_hashed":"id_count"})
output_id_Q4_sales=all_rewards_most_recent.groupby(['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'])['sales_recent_Quarter'].sum().to_frame().reset_index()

output=pd.merge(output_id_count,output_id_Q4_sales,on=['Group_before_the_Quarter','Group_after_the_Quarter','NewRewards','Recent_Quarter_Shopping_Group'],how="outer")
output


,Group_before_the_Quarter,Group_after_the_Quarter,NewRewards,Recent_Quarter_Shopping_Group,id_count,sales_recent_Quarter
0,Active,Active,Old_Rewards_Members,Recent_Quarter_No_Shop,9328339,0.000000e+00
1,Active,Active,Old_Rewards_Members,Recent_Quarter_Shopped,6329413,5.846814e+08
2,Active,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_Shopped,173,3.226110e+04
3,Active,Lapsed_12_48,Old_Rewards_Members,Recent_Quarter_No_Shop,1538186,0.000000e+00
4,Lapsed_13_48,Active,Old_Rewards_Members,Recent_Quarter_Shopped,598001,5.113585e+07
5,Lapsed_13_48,Lapsed_12_48,Old_Rewards_Members,Recent_Quarter_No_Shop,7658420,0.000000e+00
6,Lapsed_13_48,WD_48+,Old_Rewards_Members,Recent_Quarter_No_Shop,95393,0.000000e+00
7,NotAvailable_Before_Quarter,Active,Old_Rewards_Members,Recent_Quarter_Shopped,408174,4.724743e+07
8,NotAvailable_Before_Quarter,Active,Recent_Quarter_New_Sign_Ups,Recent_Quarter_Shopped,1056189,1.428588e+08
9,WD_48+,Active,Old_Rewards_Members,Recent_Quarter_Shopped,100,1.326557e+04


In [27]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration'

In [ ]:
output['Recent_Quarteer_Sale_per_ID']=output['sales_recent_Quarter']/output['id_count']
output['Recent_Quarteer_Sale_per_ID']=output['Recent_Quarteer_Sale_per_ID'].apply(lambda x: np.round(x,2))

output['inclusion/exclusion']=np.where((output['Group_before_the_Quarter']!="NotAavailable_Before_Quarter") & (output['NewRewards']=="Recent_Quarter_New_Sign_Ups"),"Exclusion","Inclusion")


In [28]:
output.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2019Q1_rewards_migration_output_summary_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [29]:
all_rewards_most_recent.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/ID_Migration/BL_2019Q1_rewards_migration_rewards_data_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [30]:
all_rewards_most_recent.shape

(27030825, 14)

In [31]:
len(all_rewards_most_recent['customer_id_hashed'].unique())

27030825